In [5]:
import pandas as pd
import numpy as np
from scipy.stats import f, fisher_exact
import math as m
%load_ext blackcellmagic

Позднее взыскание: исследуйте дисперсию размера выданных кредитов в позднем взыскании.

Используя критерий Фишера, сравните между собой:

1. Дисперсию кредитов, выданных женщинам и мужчинам (исключая строки, где пол не определён или сумма кредита отсутствует)
2. Дисперсию кредитов в разных рисковых сегментах (cl_segm). Исключите строки, где не определён клиентский сегмент или размер кредита
3. Определите три самых крупных name_city (с точки зрения каунта строк) и сравните между собой дисперсии кредитов, выданных в них.

Оформите excel файл с результатами (названия и цвета вкладок, описание того, какие результаты соответствуют какому сравнению и тд)

In [22]:
alpha = 0.05
df = pd.read_csv("late_collection_clients.csv")
df.drop(["id_city.1", "id_region", "name_region", "cellphone"], axis=1, inplace=True)
df = df.query("(gender == gender) and (amt_loan == amt_loan)")
df

Определим функцию `calculate_fisher(i, j, feature)` , которая для признака `feature`  и его конкретных значений `i`, `j` будет вычислять $\sigma^2_1$ и  $\sigma^2_2$, затем, ориентируясь на то, какое из этих значений больше, будет определять распределение Фишера. После того, как распределение Фишера задано, определяется булева переменная `test_eq` которая хранит в себе значение проверки совокупности неравенств 
$$F<F_{\frac{\alpha}{2}}(n-1, m-1)\; \lor\; F>F_{1-\frac{\alpha}{2}}(n-1, m-1) $$
Если нулевая гипотеза (о равенстве дисперсий) отвергнута, (`test_eq=True`), то проверяем далее альтернативную гипотезу о том, что $\sigma^2_1 >\sigma^2_2$, которая принимается в случае если $$F>F_{1-\alpha}(n-1, m-1)$$


In [21]:
def calculate_fisher(i, j, feature):
    sigma_1_squared = np.std(df[df[feature] == i]["amt_loan"].values) ** 2
    n_1 = len(df[df[feature] == i]["amt_loan"])
    sigma_2_squared = np.std(df[df[feature] == j]["amt_loan"].values) ** 2
    n_2 = len(df[df[feature] == j]["amt_loan"])
    if sigma_1_squared > sigma_2_squared:
        fisher_rv = f(n_1 - 1, n_2 - 1)
        f_score = sigma_1_squared / sigma_2_squared
    else:
        fisher_rv = f(n_2 - 1, n_1 - 1)
        f_score = sigma_2_squared / sigma_1_squared
    test_eq = f_score < fisher_rv.ppf(alpha / 2) or f_score > fisher_rv.ppf(
        1 - alpha / 2
    )
    if test_eq == True:
        # альтернативная гипотеза – sigma_1_squared > sigma_2_squared
        test_neq = f_score > fisher_rv.ppf(1 - alpha)
        return f_score, test_eq, test_neq
    else:
        return f_score, test_eq

*Здесь и далее утверждения ```True``` и ```False``` относятся к справедливости альтернативных гипотез*

In [23]:
print(calculate_fisher('M', 'F', 'gender'))

(1.0479258307076724, False)


Отсюда следует, что с большой долей вероятности, дисперсии двух выборок равны.

In [11]:
df = df.query('(cl_segm == cl_segm) and (amt_loan == amt_loan)')
df

,id_client,id_global,id_city,gender,married,first_time,age,is_educ,is_active,cl_segm,amt_loan,date_loan,credit_type,name_city
0,169,3996.0,1.0,M,False,0.0,69.0,1.0,1.0,2.0,110000,2021-04-08,POS-1,Москва
1,1622,1771.0,1.0,M,False,1.0,69.0,1.0,1.0,2.0,110000,2019-09-14,POS-1,Москва
2,2537,1882.0,1.0,M,False,1.0,39.0,0.0,1.0,2.0,70000,2020-03-26,POS-1,Москва
3,799,2500.0,1.0,F,False,0.0,18.0,1.0,1.0,2.0,500000,2020-08-08,POS-1,Москва
4,1901,1217.0,1.0,F,False,1.0,53.0,0.0,0.0,3.0,900000,2019-10-18,TOP-UP,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,2890,1582.0,NaN,M,False,1.0,21.0,1.0,1.0,2.0,300000,2020-08-30,CC,NaN
2989,1291,1222.0,NaN,F,False,1.0,42.0,1.0,1.0,2.0,200000,2018-06-21,RC,NaN
2992,1474,3537.0,NaN,F,False,1.0,60.0,1.0,1.0,2.0,200000,2019-03-10,POS-1,NaN
2995,932,2654.0,NaN,M,False,1.0,74.0,0.0,1.0,2.0,175000,2020-01-05,POS-2,NaN


В данном ниже словаре ключом является пара городов, дисперсии которых мы сравниваем. Кортеж значений для ключа состоит из статистики критерия Фишера, справедливость/ложность альтернативной гипотезы о неравенстве и справедливость/ложность альтернативной гипотезы о том, что $\sigma^2_1 >\sigma^2_2$. В силу симметрии, ключи, полючающиеся перестановкой элементов уже существующего ключа не рассматриваются. Так же не рассматриваются ключи, образованные парой одинаковых элементов

In [28]:
dct = {}
fisher_cl_segm = []
for i in df.cl_segm.unique():
    for j in df.cl_segm.unique():
        if i != j and (j, i) not in dct.keys():
            dct[(i, j)] = calculate_fisher(i, j, "cl_segm")
dct

{(2.0, 3.0): (1.028089945057517, False),
 (2.0, 4.0): (2.24934787010523, True, True),
 (2.0, 1.0): (1.231436334181799, False),
 (3.0, 4.0): (2.3125319281917287, True, True),
 (3.0, 1.0): (1.1977904658068665, False),
 (4.0, 1.0): (2.769928695462022, True, True)}

In [31]:
define_cities = df[['id_client', 'id_city']].groupby('id_city').count()
define_cities.sort_values('id_client', ascending=False, inplace=True)
define_cities.head()

,id_client
id_city,
2.0,441
1.0,437
6.0,174
5.0,172
4.0,161


In [30]:
cities = [2, 1, 6]
fisher_cities = {}
for i in cities:
    for j in cities:
        if i != j and  (j, i) not in fisher_cities.keys():
            fisher_cities[(i, j)] = calculate_fisher(i, j, "id_city")
fisher_cities

{(2, 1): (1.0704556416736806, False),
 (2, 6): (1.5541842192490534, True, True),
 (1, 6): (1.6636852656953538, True, True)}